<a href="https://colab.research.google.com/github/thatgeeman/pybx/blob/master/nbs/pybx_walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>⚠ Note: walkthrough for v0.1.3 ⚠
>
>run `! pip freeze | grep pybx` to see the installed version. 

# PyBx

PyBx is a simple python package to generate anchor boxes (aka default/prior boxes) for object detection
tasks. 

In [ ]:
! pip install pybx  # restart runtime if asked

In [ ]:
! pip freeze | grep pybx

# SSD for Object Detection

This walkthrough is build around the [Single-Shot Detection (SSD)](https://arxiv.org/pdf/1512.02325.pdf) algorithm. The SSD can be imagined as an encoder-decoder model architecture, where the input image is fed into a `backbone` (encoder) to generate inital features, which then goes through a series of 2D convolution layers (decoders) to perform further feature extraction/prediction tasks at each layer. For a single image, each layer in the decoder produces a total of `N x (4 + C)` predictions. Here `C` is the number of classes (plus one for `background` class) in the detection task and 4 comes from the corners of the rectangular bounding box. 

### Usage of the term Feature/Filter/Channel

Channel: RGB dimensione, also called a Filter

Feature: (W,H) of a single channel

## Example case 
For this example, we assume that our input image is a single channel image is of shape `[B, 3, 300, 300]` where `B` is the batch size. Assuming that a pretrained `VGG-16` is our model `backbone`, the output feature shape would be: `[B, 512, 37, 37]`. Meaning that, 512 channels of shape `[37, 37]` were extracted from each image in the batch. In the subsequent decoder layers, for simplicity we double the channels while halving the feature shape using `3x3` `stride=2` convolutions (except for first decoder layer where convolution is not applied). This results in the following shapes:

```python
torch.Size([-1, 512, 37, 37])  # inp from vgg-16 encoder
torch.Size([-1, 1024, 18, 18]) # first layer logits
torch.Size([-1, 2048, 8, 8])   # second layer logits
torch.Size([-1, 4096, 3, 3])   # third layer logits
```

<img src="https://lilianweng.github.io/lil-log/assets/images/SSD-box-scales.png" width="500" />

## Sample image
Image obtained from USC-SIPI Image Database. 
The USC-SIPI image database is a collection of digitized images. It is maintained primarily to support research in image processing, image analysis, and machine vision. The first edition of the USC-SIPI image database was distributed in 1977 and many new images have been added since then.

In [ ]:
! wget -q -O 'image.jpg' 'https://sipi.usc.edu/database/download.php?vol=misc&img=5.1.12'

## About anchor Boxes

We are expected to provide our models with "good" anchor (aka default/prior) boxes. Strong opinion: Our model is [only as good as the initial anchor boxes](https://towardsdatascience.com/anchor-boxes-the-key-to-quality-object-detection-ddf9d612d4f9) that we generate. Inorder to improve the coverage of our model, we tend to add additional anchor boxes of different aspect ratios. Now, for a single image, each layer in the decoder produces a total of `N x A x (4 + C)` predictions. Here `A` is the number of aspect ratios to generate additional anchor boxes.

### Task description

Our aim is to find the maximum number of anchor boxes in varying sizes `feature_szs` and aspect ratios `asp_ratios` across the entire image. We apply no filtering to get rid of low (IOU) anchors.

<img src="https://lilianweng.github.io/lil-log/assets/images/SSD-framework.png" width="600" />

In [ ]:
feature_szs = [(37,37), (18,18), (8,8), (3,3)]

In [ ]:
asp_ratios = [1/2., 1., 2.]

In [ ]:
from operator import __mul__
n_boxes = sum([__mul__(*f) for f in feature_szs]) 
print(f'minimum anchor boxes with 1 aspect ratio: {n_boxes}')
print(f'minimum anchor boxes with {len(asp_ratios)} aspect ratios: {n_boxes*len(asp_ratios)}')

# Loading an image

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import json

In [ ]:
im = Image.open("image.jpg").convert('RGB').resize([300,300])
_ = plt.imshow(im)

We also make 2 truth bounding boxes `bbox` for this image around the clock and the photoframe in `pascal voc` format:

In [ ]:
bbox = [dict(x_min=150, y_min=70, x_max=270, y_max=220, label='clock'),
        dict(x_min=10, y_min=180, x_max=115, y_max=260, label='frame'),]
bbox

Save annotations as a json file.

In [ ]:
with open('annots.json', 'w') as f:
  f.write(json.dumps(bbox))

# Using PyBx

In [ ]:
from pybx import anchor

image_sz = (300, 300, 3)  # W, H, C
feature_sz = (3, 3)       # number of features along W, H
asp_ratio = 1.            # aspect ratio of the anchor box

anchors, labels = anchor.bx(image_sz, feature_sz, asp_ratio)

To visualize the anchors:

In [ ]:
from pybx import vis

v = vis.VisBx(image_sz)
v.show(anchors, labels)

The boxes in white are the anchor boxes. We can hightlight them with a different color by looking up specific box labels.

In [ ]:
anchors.shape, labels

We see there are 16 labels and box coordinates reported by `anchor.bx()`, but we are certain that there are only 9 anchor boxes possible for our `feature_sz=3x3` and single `asp_ratio`. Out of the 16 calculated by `anchor.bx()`, 7 of them are considered `invalid` (they are not true anchor boxes) by `pybx` and are not shown or taken into account during further processing. `anchor.bx` in `v0.1.3` preserves them and their labels, but does not use them for calculations or visualisation, once instantiated as a `MultiBx`. To wrap a set of coordinates as `MultiBx`, we can use the `mbx()` method. 

In [ ]:
from pybx.basics import *

b = mbx(anchors, labels)  # instantiates MultiBx for us

In [ ]:
type(b)

We can iterate over a `MultiBx` object using list comprehension to understand the internal checks:

In [ ]:
[(i, b_.valid()) for i, b_ in enumerate(b)]  # only valid boxes shown

`b_.valid()` returned `True` meaning that the box is considered valid. 

We can also calculate the areas of these boxes.

Each box `b_` of the `MultiBx` b is of type `BaseBx` which has some additional methods.

In [ ]:
[b_.area() for b_ in b] 

Displaying the coordinates of the valid boxes:

In [ ]:
[b_.coords for b_ in b]  # selected boxes only!

Displaying the labels of valid boxes

In [ ]:
[b_.label for b_ in b]  # selected boxes only!

We can ofcourse see all the 16 boxes calculated by `anchor.bx()` from the `MultiBx` as well:

In [ ]:
b.coords, b.label

> The `vis.VisBx` internally converts all coordinates in list/json/ndarray to a `MultiBx` and shows only `valid` boxes. 

We can also overlay the features generated by the model on the original image. `logits=True` generates random logits (`np.random.randn`) of the same shape as feature sizes for illustration purposes. 

To aid the explainability of the model, actual model logits can also be passed into the same parameter as an array or detached tensor.

In [ ]:
# ask VisBx to use random logits with logits=True
vis.VisBx(image_sz, logits=True, feature_sz=feature_sz).show(anchors, labels)

In [ ]:
# ask VisBx to use passed logits with logits=logits
logits = np.random.randn(3,3)  # assuming these are model logits
v = vis.VisBx(image_sz, logits=logits).show(anchors, labels)

We can hightlight them with a different color if needed. Anchor boxes generated with `named=True` parameter automatically sets the label for each box in the format: `{anchor_sfx}_{feature_sz}_{asp_ratio}_{box_number}`. `anchor_sfx` is also an optional parameter that can be passed to `anchor.bx()`. Here we change the color of one anchor box and one ground truth box.

In [ ]:
labels[4]

In [ ]:
v = vis.VisBx(image_sz)
v.show(anchors, labels, color={'a_3x3_0.5_4':'red', 'clock':'orange'})

# Working with mulitple feature sizes and aspect ratios
Finally we calculate anchor boxes for multiple feature sizes and aspect ratios.

In [ ]:
feature_szs = [(3, 3), (2, 2)]
asp_ratios = [1/2., 2.]

anchors, labels = anchor.bxs(image_sz, feature_szs, asp_ratios)

This is essentially a wrapper to do list comprehension over the passed feature sizes and aspect ratios (but additionally stacks them together into an ndarray).

```
[anchor.bx(image_sz, f, ar) for f in feature_szs for ar in asp_ratios]
```

In [ ]:
labels[4], labels[32]

In [ ]:
v = vis.VisBx(image_sz)
v.show(anchors, labels, color={'a_3x3_0.5_4':'red', 'a_2x2_0.5_0':'red'})

As simple as that! Do leave a star or raise issues and suggestions on the project page if you found this useful! 

Project page: [GitHub](https://github.com/thatgeeman/pybx) 

PyPi Package: [PyBx](https://pypi.org/project/pybx/)